![x](https://zdnet4.cbsistatic.com/hub/i/r/2017/12/17/e9b8f576-8c65-4308-93fa-55ee47cdd7ef/resize/370xauto/30f614c5879a8589a22e57b3108195f3/databricks-logo.png)

&copy; 2019 Databricks, Inc. All rights reserved.<br/>

# Model Serving

Once we have our Data Science teams producing a model, we would like to be able to use this in our day to day activities, in order to create value to our business units.

In order to do so, we may have the following scenarios:
* Ad-hoc usage
* A scheduled job that might be running once a day / week / month
* A continuous streaming job that needs to score new items non-stop
* A model endpoint that needs to return a score when provided with data

In [4]:
%fs ls /mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/

path,name,size
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/malware_sample_sub.csv,malware_sample_sub.csv,290570393
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/malware_test.csv,malware_test.csv,3795687226
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/malware_train.csv,malware_train.csv,4384966482
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_stream_merged/,workshop_stream_merged/,0
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_stream_merged_sample/,workshop_stream_merged_sample/,0
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_train.csv/,workshop_train.csv/,0
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_train_clean_delta/,workshop_train_clean_delta/,0
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_train_clean_delta_sample/,workshop_train_clean_delta_sample/,0
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_train_enrich.csv/,workshop_train_enrich.csv/,0
dbfs:/mnt/databricks-workshop-datasets/End-to-End-ML-Lifecycle/workshop_train_enrich_sample.csv/,workshop_train_enrich_sample.csv/,0


In [5]:
#Firstly, let's load our validation set (generated in the previous notebook) into a pyspark dataframe

df = spark.read.format("delta").load("/databricks_workshop/validate_clean")

In [6]:
display(df)

MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
34d9befae008da189a253a86b56b9b68,0,7,0,53447,1,1,1,171,96401,18,211,182,15063,768,1,0,0,108,1,1,2941,263902,4,5,2560,186609,0,8192,1920,1080,15063,1206,29,125,0,0,142,32235,0,0,1,0,0,1,3
34ec3b904805cdf02eb7df9acda9d891,0,7,0,53447,1,1,1,107,141940,27,138,134,17134,768,1,0,0,137,1,1,4142,129104,8,5,2737,104991,0,4096,1920,1080,17134,191,20,83,0,0,142,51578,1,0,0,0,0,0,1
31adc6f10414ea4a231d870d7af97b48,0,7,0,43856,2,1,1,141,116277,27,167,227,16299,768,1,0,0,117,1,1,2102,248395,4,5,3396,454575,0,4096,1366,768,16299,579,10,35,0,0,554,33139,1,0,1,1,0,0,10
30fa1eb28798f3e5d2317003bfd1be8e,0,7,0,46545,2,1,1,51,80810,27,98,103,14393,768,1,0,0,94,1,1,2668,171197,4,1,198,908098,0,4096,1600,900,14393,693,5,26,0,0,628,27770,1,0,0,0,0,0,6
3291e672f5238862bd332bf9a59a163c,0,7,0,53447,1,1,1,195,10222,27,258,75,15063,768,1,0,0,108,1,1,2102,34020,4,5,3389,232939,0,4096,1600,900,15063,966,8,31,0,0,93,33142,1,0,0,0,0,0,1
73e2650cdab9834ce9de1b4eadaf31ca,0,7,0,53300,2,1,1,43,71228,27,53,42,10586,768,1,0,0,87,1,1,1443,256685,4,5,3063,51845,0,8192,1920,1080,10586,753,37,158,0,0,355,4319,1,0,0,0,0,0,7
7097d737073cbd869056147eea42ba1b,0,7,0,53447,1,1,1,207,37252,27,277,75,15063,768,1,0,0,107,1,1,1443,256580,2,5,3426,295815,0,4096,1366,768,15063,674,8,31,0,0,355,19946,1,0,0,0,0,0,13
b31f0c53fbc391b746b0114a6d4136de,0,7,0,53447,1,1,1,68,114574,27,277,75,10240,256,1,0,0,64,1,1,2206,244535,4,5,3392,349904,0,4096,1366,768,10240,17889,8,31,0,0,554,33041,0,0,1,0,0,0,12
b31ee6eaaf62a81402099dfaf72b3568,0,7,0,47238,2,1,1,29,143673,18,35,171,17134,768,1,0,0,137,1,1,4142,170823,4,5,2701,939052,0,8192,1920,1080,17134,48,26,119,0,0,142,52904,1,0,0,0,0,0,10
f17926a89a60083410a3832b0fe5b638,0,7,0,53447,1,1,1,141,5299,27,167,227,16299,256,1,0,0,117,1,1,2668,170931,4,5,3394,437080,0,2048,1366,768,16299,431,9,34,0,0,628,21628,1,0,0,0,0,0,10


In [7]:
df.count()

Out[ 3 ]: 462

### Scoring via UDFs in Python and SQL

In [9]:
#Let's also load in our sklearn mlflow model
import mlflow
import mlflow.spark

#Also load it as a UDF
pyfunc_udf = mlflow.pyfunc.spark_udf(spark, model_uri="dbfs:/databricks/mlflow/1622123275403635/25b2a916f8ab4879b267e443a7f6f0b4/artifacts/lrModelSK")

#And register it to SQL
spark.udf.register("scoreUDF", pyfunc_udf)

Out[ 4 ]: <function mlflow.pyfunc.spark_udf.<locals>.predict>

In [10]:
#Score using a python UDF

df_pyudf = df.withColumn("prediction", pyfunc_udf('IsBeta',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'HasTpm',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsBuild',
 'OsSuite',
 'IsProtected',
 'AutoSampleOptIn',
 'SMode',
 'IeVerIdentifier',
 'Firewall',
 'UacLuaenable',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalRAM',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_IsPortableOperatingSystem',
 'Census_IsFlightsDisabled',
 'Census_FirmwareManufacturerIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Census_IsSecureBootEnabled',
 'Census_IsVirtualDevice',
 'Census_IsTouchEnabled',
 'Census_IsPenCapable',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer',
 'Wdft_RegionIdentifier'))

In [11]:
display(df_pyudf)

MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,prediction
34d9befae008da189a253a86b56b9b68,0,7,0,53447,1,1,1,171,96401,18,211,182,15063,768,1,0,0,108,1,1,2941,263902,4,5,2560,186609,0,8192,1920,1080,15063,1206,29,125,0,0,142,32235,0,0,1,0,0,1,3,1.0
34ec3b904805cdf02eb7df9acda9d891,0,7,0,53447,1,1,1,107,141940,27,138,134,17134,768,1,0,0,137,1,1,4142,129104,8,5,2737,104991,0,4096,1920,1080,17134,191,20,83,0,0,142,51578,1,0,0,0,0,0,1,1.0
31adc6f10414ea4a231d870d7af97b48,0,7,0,43856,2,1,1,141,116277,27,167,227,16299,768,1,0,0,117,1,1,2102,248395,4,5,3396,454575,0,4096,1366,768,16299,579,10,35,0,0,554,33139,1,0,1,1,0,0,10,0.0
30fa1eb28798f3e5d2317003bfd1be8e,0,7,0,46545,2,1,1,51,80810,27,98,103,14393,768,1,0,0,94,1,1,2668,171197,4,1,198,908098,0,4096,1600,900,14393,693,5,26,0,0,628,27770,1,0,0,0,0,0,6,1.0
3291e672f5238862bd332bf9a59a163c,0,7,0,53447,1,1,1,195,10222,27,258,75,15063,768,1,0,0,108,1,1,2102,34020,4,5,3389,232939,0,4096,1600,900,15063,966,8,31,0,0,93,33142,1,0,0,0,0,0,1,1.0
73e2650cdab9834ce9de1b4eadaf31ca,0,7,0,53300,2,1,1,43,71228,27,53,42,10586,768,1,0,0,87,1,1,1443,256685,4,5,3063,51845,0,8192,1920,1080,10586,753,37,158,0,0,355,4319,1,0,0,0,0,0,7,1.0
7097d737073cbd869056147eea42ba1b,0,7,0,53447,1,1,1,207,37252,27,277,75,15063,768,1,0,0,107,1,1,1443,256580,2,5,3426,295815,0,4096,1366,768,15063,674,8,31,0,0,355,19946,1,0,0,0,0,0,13,1.0
b31f0c53fbc391b746b0114a6d4136de,0,7,0,53447,1,1,1,68,114574,27,277,75,10240,256,1,0,0,64,1,1,2206,244535,4,5,3392,349904,0,4096,1366,768,10240,17889,8,31,0,0,554,33041,0,0,1,0,0,0,12,0.0
b31ee6eaaf62a81402099dfaf72b3568,0,7,0,47238,2,1,1,29,143673,18,35,171,17134,768,1,0,0,137,1,1,4142,170823,4,5,2701,939052,0,8192,1920,1080,17134,48,26,119,0,0,142,52904,1,0,0,0,0,0,10,1.0
f17926a89a60083410a3832b0fe5b638,0,7,0,53447,1,1,1,141,5299,27,167,227,16299,256,1,0,0,117,1,1,2668,170931,4,5,3394,437080,0,2048,1366,768,16299,431,9,34,0,0,628,21628,1,0,0,0,0,0,10,1.0


Let's also load the data into a sql table

In [13]:
%sql

DROP TABLE IF EXISTS df_sqludf;
CREATE TABLE df_sqludf
USING DELTA
LOCATION "/databricks_workshop/validate_clean"

And use our UDF in sql

In [15]:
%sql

SELECT MachineIdentifier, scoreUDF(`IsBeta`,
`RtpStateBitfield`,
`IsSxsPassiveMode`,
`AVProductStatesIdentifier`,
`AVProductsInstalled`,
`AVProductsEnabled`,
`HasTpm`,
`CountryIdentifier`,
`CityIdentifier`,
`OrganizationIdentifier`,
`GeoNameIdentifier`,
`LocaleEnglishNameIdentifier`,
`OsBuild`,
`OsSuite`,
`IsProtected`,
`AutoSampleOptIn`,
`SMode`,
`IeVerIdentifier`,
`Firewall`,
`UacLuaenable`,
`Census_OEMNameIdentifier`,
`Census_OEMModelIdentifier`,
`Census_ProcessorCoreCount`,
`Census_ProcessorManufacturerIdentifier`,
`Census_ProcessorModelIdentifier`,
`Census_SystemVolumeTotalCapacity`,
`Census_HasOpticalDiskDrive`,
`Census_TotalPhysicalRAM`,
`Census_InternalPrimaryDisplayResolutionHorizontal`,
`Census_InternalPrimaryDisplayResolutionVertical`,
`Census_OSBuildNumber`,
`Census_OSBuildRevision`,
`Census_OSInstallLanguageIdentifier`,
`Census_OSUILocaleIdentifier`,
`Census_IsPortableOperatingSystem`,
`Census_IsFlightsDisabled`,
`Census_FirmwareManufacturerIdentifier`,
`Census_FirmwareVersionIdentifier`,
`Census_IsSecureBootEnabled`,
`Census_IsVirtualDevice`,
`Census_IsTouchEnabled`,
`Census_IsPenCapable`,
`Census_IsAlwaysOnAlwaysConnectedCapable`,
`Wdft_IsGamer`,
`Wdft_RegionIdentifier`) AS prediction FROM df_sqludf

MachineIdentifier,prediction
34d9befae008da189a253a86b56b9b68,1.0
34ec3b904805cdf02eb7df9acda9d891,1.0
31adc6f10414ea4a231d870d7af97b48,0.0
30fa1eb28798f3e5d2317003bfd1be8e,1.0
3291e672f5238862bd332bf9a59a163c,1.0
73e2650cdab9834ce9de1b4eadaf31ca,1.0
7097d737073cbd869056147eea42ba1b,1.0
b31f0c53fbc391b746b0114a6d4136de,0.0
b31ee6eaaf62a81402099dfaf72b3568,1.0
f17926a89a60083410a3832b0fe5b638,1.0


### Scoring in Batch

The most common way of using Data Science models today is by using a batch scoring system. For example, let's say you have implemented a churn system, and every morning you'd like one of your colleagues to go through a list of the top 5 most likely to churn customers, and ring them up. In the backend, that usually means that, the evening before, after the day's data has been collected, a ML model runs on the new information to produce a list of 5 likely churners.

In our case, we'd like to run our model once against our test dataset, to find out which of our machines are likely to have malware.

In [17]:
import mlflow
import mlflow.spark

#Load the ML lib model as a spark object
model = mlflow.spark.load_model(model_uri = "dbfs:/databricks/mlflow/1622123275403635/7ecee1d76b6242d5a326e10e7e55e679/artifacts/lrModel")

In [18]:
scoredDF = model.transform(df)

In [19]:
display(scoredDF)

MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,featureVector,features,rawPrediction,probability,prediction
34d9befae008da189a253a86b56b9b68,0,7,0,53447,1,1,1,171,96401,18,211,182,15063,768,1,0,0,108,1,1,2941,263902,4,5,2560,186609,0,8192,1920,1080,15063,1206,29,125,0,0,142,32235,0,0,1,0,0,1,3,"List(1, 45, List(), List(0.0, 7.0, 0.0, 53447.0, 1.0, 1.0, 1.0, 171.0, 96401.0, 18.0, 211.0, 182.0, 15063.0, 768.0, 1.0, 0.0, 0.0, 108.0, 1.0, 1.0, 2941.0, 263902.0, 4.0, 5.0, 2560.0, 186609.0, 0.0, 8192.0, 1920.0, 1080.0, 15063.0, 1206.0, 29.0, 125.0, 0.0, 0.0, 142.0, 32235.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 3.0))","List(1, 45, List(), List(0.0, 6.730820767069861, 0.0, 3.9612283578787593, 1.901942338875175, 6.078670516638148, 22.287172223935567, 2.68723634822555, 1.9558888353453296, 3.224426971232029, 2.3561865659921377, 2.562563058728277, 7.874822881914021, 3.135049418312507, 5.030367141153888, 0.0, 0.0, 3.1723865652261685, 7.187101054396513, 4.079769127460697, 2.248156117049332, 3.617844618353276, 1.9173753063486916, 3.819808505284234, 3.0279469183788783, 0.5699486290697847, 0.0, 1.6732123444817417, 5.207986976919206, 4.9585320340887655, 8.465486021127058, 0.4352284346057906, 2.769621760712916, 2.7124410929172367, 0.0, 0.0, 0.6361069741862515, 1.5143468326727103, 0.0, 0.0, 2.949384118462997, 0.0, 0.0, 2.193257584030426, 0.6666118682199336))","List(1, 2, List(), List(-0.3534155394379736, 0.3534155394379736))","List(1, 2, List(), List(0.41255440743564964, 0.5874455925643504))",1.0
34ec3b904805cdf02eb7df9acda9d891,0,7,0,53447,1,1,1,107,141940,27,138,134,17134,768,1,0,0,137,1,1,4142,129104,8,5,2737,104991,0,4096,1920,1080,17134,191,20,83,0,0,142,51578,1,0,0,0,0,0,1,"List(1, 45, List(), List(0.0, 7.0, 0.0, 53447.0, 1.0, 1.0, 1.0, 107.0, 141940.0, 27.0, 138.0, 134.0, 17134.0, 768.0, 1.0, 0.0, 0.0, 137.0, 1.0, 1.0, 4142.0, 129104.0, 8.0, 5.0, 2737.0, 104991.0, 0.0, 4096.0, 1920.0, 1080.0, 17134.0, 191.0, 20.0, 83.0, 0.0, 0.0, 142.0, 51578.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0))","List(1, 45, List(), List(0.0, 6.730820767069861, 0.0, 3.9612283578787593, 1.901942338875175, 6.078670516638148, 22.287172223935567, 1.6814870716966894, 2.879833832521614, 4.836640456848043, 1.5410130147247156, 1.8867222520307094, 8.957526074401834, 3.135049418312507, 5.030367141153888, 0.0, 0.0, 4.0242311058887505, 7.187101054396513, 4.079769127460697, 3.1662232699144277, 1.7698926556368704, 3.834750612697383, 3.819808505284234, 3.2373010607824177, 0.3206676875963419, 0.0, 0.8366061722408709, 5.207986976919206, 4.9585320340887655, 9.629399023168759, 0.06892921310920895, 1.9100839729054595, 1.8010608856970451, 0.0, 0.0, 0.6361069741862515, 2.42304888895899, 2.000488887417364, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22220395607331117))","List(1, 2, List(), List(-0.4475366745457567, 0.4475366745457567))","List(1, 2, List(), List(0.38994660311675705, 0.610053396883243))",1.0
31adc6f10414ea4a231d870d7af97b48,0,7,0,43856,2,1,1,141,116277,27,167,227,16299,768,1,0,0,117,1,1,2102,248395,4,5,3396,454575,0

### Stream Scoring

Another option of scoring would be through real-time streams. Imagine a scenario where you have sensor data from different machinery coming in, and, in a predictive maintenance situation, you'd like to be notified on the first occurence of a sensor being out of bounds, to minimise repair times and costs. Due to the throughput of that type of information, a Spark Streaming job is ideal to score data on-the-go, and showcase any potential anomalies.

While in our scenario we don't have frequent updates of information, we can still leverage Spark Streaming to score our batch dataset. We only have to read it as a stream!

In [21]:
import mlflow
import mlflow.spark

#Load the model as a spark object
model = mlflow.spark.load_model(model_uri = "dbfs:/databricks/mlflow/1622123275403635/7ecee1d76b6242d5a326e10e7e55e679/artifacts/lrModel")

In [22]:
df_stream = spark.readStream.format("delta").load("/databricks_workshop/cleanprep_delta_test")

In [23]:
scoredStream = model.transform(df_stream)

In [24]:
display(scoredStream)

MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,featureVector,features,rawPrediction,probability,prediction
e83dccddc552a5472537672a0d29c7fa,0,7,0,52076,2,1,1,122,25179,27,89,88,17134,256,1,0,0,137,0,1,2102,238620,4,5,3063,182771,0,8192,1920,1080,17134,285,14,49,0,0,554,33015,0,0,0,0,0,0,11,"List(1, 45, List(), List(0.0, 7.0, 0.0, 52076.0, 2.0, 1.0, 1.0, 122.0, 25179.0, 27.0, 89.0, 88.0, 17134.0, 256.0, 1.0, 0.0, 0.0, 137.0, 0.0, 1.0, 2102.0, 238620.0, 4.0, 5.0, 3063.0, 182771.0, 0.0, 8192.0, 1920.0, 1080.0, 17134.0, 285.0, 14.0, 49.0, 0.0, 0.0, 554.0, 33015.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0))","List(1, 45, List(), List(0.0, 6.730820767069861, 0.0, 3.8596165914811733, 3.80388467775035, 6.078670516638148, 22.287172223935567, 1.9172095583831412, 0.5108590676980536, 4.836640456848043, 0.9938417268876789, 1.2390414789455404, 8.957526074401834, 1.0450164727708355, 5.030367141153888, 0.0, 0.0, 4.0242311058887505, 0.0, 4.079769127460697, 1.6068086222501514, 3.2712525211307937, 1.9173753063486916, 3.819808505284234, 3.6228911761697282, 0.5582264568360241, 0.0, 1.6732123444817417, 5.207986976919206, 4.9585320340887655, 9.629399023168759, 0.10285249076505001, 1.3370587810338217, 1.0632769084235567, 0.0, 0.0, 2.481713124642136, 1.5509899389076944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4442435168064227))","List(1, 2, List(), List(0.32227311271458897, -0.32227311271458897))","List(1, 2, List(), List(0.579878127026892, 0.420121872973108))",0.0
e83dce90f24bb2001cad04f5a4022e73,0,7,0,53447,1,1,1,141,56274,27,167,227,16299,256,1,0,0,117,1,1,2206,200340,2,1,326,111739,0,2048,1280,800,16299,431,9,34,0,0,500,33032,0,0,0,0,0,0,10,"List(1, 45, List(), List(0.0, 7.0, 0.0, 53447.0, 1.0, 1.0, 1.0, 141.0, 56274.0, 27.0, 167.0, 227.0, 16299.0, 256.0, 1.0, 0.0, 0.0, 117.0, 1.0, 1.0, 2206.0, 200340.0, 2.0, 1.0, 326.0, 111739.0, 0.0, 2048.0, 1280.0, 800.0, 16299.0, 431.0, 9.0, 34.0, 0.0, 0.0, 500.0, 33032.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0))","List(1, 45, List(), List(0.0, 6.730820767069861, 0.0, 3.9612283578787593, 1.901942338875175, 6.078670516638148, 22.287172223935567, 2.2157913748526465, 1.1417484084213139, 4.836640456848043, 1.8648490830364313, 3.1961638150072464, 8.520994367145763, 1.0450164727708355, 5.030367141153888, 0.0, 0.0, 3.436752112328349, 7.187101054396513, 4.079769127460697, 1.6863081925232322, 2.746470245928016, 0.9586876531743458, 0.7639617010568468, 0.38559011538731025, 0.34127769755814924, 0.0, 0.41830308612043543, 3.4719913179461375, 3.672986691917604, 9.160124587290042, 0.15554183691135634, 0.8595377878074567, 0.7377839772734884, 0.0, 0.0, 2.2398132893882092, 1.5517885707102517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2220395607331116))","List(1, 2, List(), List(-0.13366822764313535, 0.13366822764313535))","List(1, 2, List(), List(0.4666326101063139, 0.5333673898936862))",1.0
e83dd03d4d9f89435f4659076a8d380a,0,7,0,53447,1,1,1,39,115403,27,252,106,17134,768,1,0,0,137,1,1,2206,240319,4,5,2529,475583,0,8192,1366,768,17134,286,5,26,0,0,500,14767,0,0,0,0,0,0,

Here, MLFlow has allowed us an easy way to leverage our model for scoring, Spark has enabled us to do it at scale, and Databricks has provided the platform and infrastructure needed to enable the use case!

## Next Step

[Serving via REST API]($./3-03 Serving via REST API)

&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>